Bước 1: Cài và import thư viện cần thiết

In [2]:
import web3
import solcx
import json

In [3]:
# 1. Kiểm tra phiên bản web3
print("Web3 version:", web3.__version__)

Web3 version: 7.10.0


In [4]:
print("Các phiên bản Solidity có sẵn:", solcx.get_installed_solc_versions())

Các phiên bản Solidity có sẵn: [<Version('0.8.0')>]


In [5]:
print("Các phiên bản có thể cài:", solcx.get_installable_solc_versions())

Các phiên bản có thể cài: [<Version('0.8.29')>, <Version('0.8.28')>, <Version('0.8.27')>, <Version('0.8.26')>, <Version('0.8.25')>, <Version('0.8.24')>, <Version('0.8.23')>, <Version('0.8.22')>, <Version('0.8.21')>, <Version('0.8.20')>, <Version('0.8.19')>, <Version('0.8.18')>, <Version('0.8.17')>, <Version('0.8.16')>, <Version('0.8.15')>, <Version('0.8.14')>, <Version('0.8.13')>, <Version('0.8.12')>, <Version('0.8.11')>, <Version('0.8.10')>, <Version('0.8.9')>, <Version('0.8.8')>, <Version('0.8.7')>, <Version('0.8.6')>, <Version('0.8.5')>, <Version('0.8.4')>, <Version('0.8.3')>, <Version('0.8.2')>, <Version('0.8.1')>, <Version('0.8.0')>, <Version('0.7.6')>, <Version('0.7.5')>, <Version('0.7.4')>, <Version('0.7.3')>, <Version('0.7.2')>, <Version('0.7.1')>, <Version('0.7.0')>, <Version('0.6.12')>, <Version('0.6.11')>, <Version('0.6.10')>, <Version('0.6.9')>, <Version('0.6.8')>, <Version('0.6.7')>, <Version('0.6.6')>, <Version('0.6.5')>, <Version('0.6.4')>, <Version('0.6.3')>, <Version('

Bước 2: Cài đặt Solidity Compiler 0.8.0

In [14]:
from solcx import install_solc

# Cài đặt phiên bản Solidity 0.8.0 (nếu chưa có)
install_solc('0.8.0')

<Version('0.8.0')>

In [16]:
print(" Phiên bản đã cài:", solcx.get_installed_solc_versions())

 Phiên bản đã cài: [<Version('0.8.0')>]


Bước 3: Đọc mã hợp đồng từ file.sol và biên dịch

Mục tiêu:
1. Đọc nội dung file Message.sol
2. Biên dịch bằng compiler v0.8.0
3. Thu được: compiled_sol, abi, bytecode

3.1. Đọc nội dung file.sol

In [17]:
with open("Message.sol", "r") as file:
    source_code = file.read()

# Kiểm tra xem đã đọc đúng chưa
print(source_code[:200]) # In thử 200 ký tự đầu

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract MessageContract {
    string public message;

    constructor(string memory _message) {
        message = _message;
    }

    functio


3.2. Biên dịch hợp đồng bằng compile_standard()

Mục tiêu:
1. Biên dịch mã Solidity đã đọc
2. Tạo object compiled_sol chứa toàn bộ ABI + bytecode + metadata

- compiled_sol là một dictionary (kiểu dict trong Python) chứa toàn bộ kết quả biên dịch từ source Solidity được cung cấp.

In [18]:
from solcx import compile_standard
compiled_sol = compile_standard({
    "language": "Solidity",
    "sources": {
        "Message.sol": {
            "content": source_code
        }
    },
    "settings": {
        "outputSelection": {
            "*": {
                "*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]
            }
        }
    }
}, solc_version="0.8.0")


In [19]:
compiled_sol

{'contracts': {'Message.sol': {'MessageContract': {'abi': [{'inputs': [{'internalType': 'string',
        'name': '_message',
        'type': 'string'}],
      'stateMutability': 'nonpayable',
      'type': 'constructor'},
     {'inputs': [],
      'name': 'message',
      'outputs': [{'internalType': 'string', 'name': '', 'type': 'string'}],
      'stateMutability': 'view',
      'type': 'function'},
     {'inputs': [{'internalType': 'string',
        'name': '_newMessage',
        'type': 'string'}],
      'name': 'updateMessage',
      'outputs': [],
      'stateMutability': 'nonpayable',
      'type': 'function'}],
    'evm': {'bytecode': {'generatedSources': [{'ast': {'nodeType': 'YulBlock',
         'src': '0:2705:1',
         'statements': [{'body': {'nodeType': 'YulBlock',
            'src': '102:258:1',
            'statements': [{'nodeType': 'YulAssignment',
              'src': '112:74:1',
              'value': {'arguments': [{'arguments': [{'name': 'length',
              

3.3 – Trích xuất ABI và Bytecode

LẤY ABI VÀ BYTECODE TỪ KẾT QUẢ BIÊN DỊCH

Mục tiêu:

Từ biến compiled_sol, ta lấy ra:

abi: mô tả giao diện các hàm để tương tác

bytecode: mã máy để triển khai lên Ethereum

In [20]:
abi = compiled_sol['contracts']['Message.sol']['MessageContract']['abi']
bytecode = compiled_sol['contracts']['Message.sol']['MessageContract']['evm']['bytecode']['object']

In [21]:
print(" ABI (rút gọn):", abi[:1])
print(" Bytecode (rút gọn):", bytecode[:20], "...")

 ABI (rút gọn): [{'inputs': [{'internalType': 'string', 'name': '_message', 'type': 'string'}], 'stateMutability': 'nonpayable', 'type': 'constructor'}]
 Bytecode (rút gọn): 60806040523480156100 ...


Bước 4: Kết nối ganache và triển khai hợp đồng



Mục tiêu:

Kết nối với Ganache đang chạy tại http://127.0.0.1:7545

Triển khai MessageContract với constructor nhận chuỗi "Chào Anh/Chị!"



In [30]:
# Kết nối tới Ganache GUI (Quickstart mode)
from web3 import Web3

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))

# Kiểm tra kết nối
print("Kết nối thành công? ->", w3.is_connected)

# Lấy danh sách tài khoản có sẵn trong Ganache
accounts = w3.eth.accounts
w3.eth.default_account = accounts[1]
print("Tài khoản mặc định:", w3.eth.default_account)

Kết nối thành công? -> <bound method Web3.is_connected of <web3.main.Web3 object at 0x00000222BE07FE50>>
Tài khoản mặc định: 0xeB8D28b885CA2Bd3A6afF4a2A1f22aF0d4E330f7


4.2 – TRIỂN KHAI MessageContract lên Ganache

Mục tiêu:

Dùng ABI + Bytecode đã biên dịch

Gửi giao dịch triển khai contract với thông điệp khởi tạo "Chào Quý khách!"

In [32]:
# Tạo đối tượng contract từ ABI và Bytecode
MessageContract = w3.eth.contract(abi=abi, bytecode=bytecode)

# Gửi giao dịch triển khai contract, truyền constructor message
tx_hash = MessageContract.constructor("Chào Quý khách!").transact()

# Chờ nhận kết quả giao dịch để lấy địa chỉ contract
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# In địa chỉ contract vừa triển khai
contract_address = tx_receipt.contractAddress
print("Hợp đồng đã được triển khai tại địa chỉ:", contract_address)

Hợp đồng đã được triển khai tại địa chỉ: 0xF8445EE06F988eD3123321aB287efE9cd4797a7f


In [33]:
print("contract_address", contract_address)

contract_address 0xF8445EE06F988eD3123321aB287efE9cd4797a7f


Bước 5: Tương tác hợp đồng sau khi triển khai

Mục tiêu: Gọi biến message() để xem nội dung và cập nhật bằng updatemassage()

In [37]:
# 5.1 - Tạo instance từ địa chỉ hợp đồng đã triển khai
contract_address = tx_receipt.contractAddress
contract_instance = w3.eth.contract(address=contract_address, abi=abi)

# 5.2 - Đọc giá trị ban đầu (getter tự động từ biến public)
print('Tin nhắn ban đầu:', contract_instance.functions.message().call())

# 5.3 - Cập nhật tin nhắn mới bằng hàm updateMessage()
tx_hash = contract_instance.functions.updateMessage('Xin chào từ Web3.py!').transact({
    'from': w3.eth.accounts[0]
})
w3.eth.wait_for_transaction_receipt(tx_hash)

# 5.4 - Kiểm tra lại sau khi cập nhật
print('Tin nhắn sau khi cập nhật:', contract_instance.functions.message().call())

Tin nhắn ban đầu: Chào Quý khách!
Tin nhắn sau khi cập nhật: Xin chào từ Web3.py!
